In [21]:
from pathlib import Path
import os

from local.constants import WORKSPACE_ROOT

### the following will likely need fidgeting of paths to run properly, but calls to specific tools and post/pre processsing are correct 

In [22]:
# https://github.com/rrwick/Filtlong
_out = WORKSPACE_ROOT/"data/Epi300_RC.qc.fastq.gz"
if not _out.exists():
    os.system(f"""\
        filtlong --min_length 1000 --keep_percent 90 --target_bases 500000000 {WORKSPACE_ROOT}/data/Epi300_RC.fastq.gz | pigz > {out}
    """)

In [23]:
DATA = WORKSPACE_ROOT/"data"
FASTQ = DATA/"work/Epi300_RC.qc.fastq"

if not FASTQ.exists():
    os.system(f"""\
        pigz -dc {WORKSPACE_ROOT}/data/Epi300_RC.qc.fastq.gz > {FASTQ}
    """)

In [24]:
container = WORKSPACE_ROOT/"lib/containers/goldrush.sif"
ASSEMBLY_DIR = WORKSPACE_ROOT/"data/assembly"
ASSEMBLY_DIR.mkdir(exist_ok=True)
epi300 = "epi300"
cpus = 14

In [8]:
size = "5e6"

_gr_result_file = "assembly_golden_path.goldpolish-polished.span2.dist500.tigmint.fa.k40.w250.ntLink-5rounds.fa"
_fq_no_ext = FASTQ.stem
cmd = f"goldrush run reads={_fq_no_ext} G={size} t={cpus} p=assembly -B"
os.system(f"""\
    cd {WORKSPACE_ROOT}/data/work
    echo {cmd}
    apptainer run {container} {cmd}
    
    cp -L ./{_gr_result_file} {ASSEMBLY_DIR}/{epi300}.goldrush.fna
    mkdir {epi300}
    mv goldrush_intermediate_files ./{epi300}
    mv *.fa ./{epi300}
    tar -cf - ./{epi300} | pigz -7 -p {cpus} >{ASSEMBLY_DIR}/{epi300}.tar.gz
    rm -r ./{epi300}
""")

goldrush run reads=Epi300_RC.qc G=5e6 t=14 p=assembly -B


INFO:    underlay of /etc/localtime required more than 50 (75) bind mounts


mkdir -p goldrush_intermediate_files
cd goldrush_intermediate_files && ln -sf ../Epi300_RC.qc.fastq && goldrush run-in-dir reads=Epi300_RC.qc G=5e6 t=14 z=1000 track_time=0 k=22 w=16 tile=1000 b=10 u=5 a=1 o=0.1 x=10 h=3 s=1011011110110111101101 m=20000 M=5 r=0.9 P=15 d=5 span=2 dist=500 k_ntLink=40 w_ntLink=250 rounds=5 polisher=goldpolish polisher_mapper=minimap2 shared_mem=/dev/shm
make[1]: Entering directory '/home/tony/workspace/projects/epi300_assembly/data/work/goldrush_intermediate_files'
goldrush-path  -k 22 -w 16 -t 1000 -u 5 -a 1 -o 0.1 -p assembly_silver_path -i Epi300_RC.qc.fastq  -h 3 -j 14 -x10 -P 15 -d 5 -s 1011011110110111101101 -g 5e6 -b 10 -r 0.9 --silver_path -M 5 -m 20000 --verbose


Using preset spaced seed
with:
	span: 22
	weight: 16
Calculating 5 silver path(s)
Using:
	tile length: 1000
	block size: 10
	seed patterns: 3
	threshold: 10
	base seed pattern: 1011011110110111101101
	minimum unassigned tiles: 5
	maximum assigned tiles: 1
	expected hash space: 15000000
	minimum average phred quality score: 15
	maximum average phred delta between first and second half of read: 5
	occupancy: 0.1
	jobs: 14
allocating bit vector
m_filterSize: 142368384
finished allocating bit vector
in 0.0050
opening: Epi300_RC.qc.fastq
inserting bit vector
num_passed_reads: 7886
num_reads: 26287
num_reads - num_passed_reads: 18401
num_reads - num_passed_reads / num_reads: 0.0000
num_reads_skipped_by_phred: 0
num_reads_skipped_by_delta: 0
num_reads_skipped_by_length: 18401
Total reads skipped: 18401
finished inserting bit vector
in 10.5861
assigning tiles
Visited 1184 reads to generate 1 silver paths
Visited 2334 reads to generate 2 silver paths
processed 10000 reads
Visited 3743 reads to 

cat assembly_silver_path_*.fq > assembly_silver_path_all.fq
goldrush-path  -k 22 -w 16 -t 1000 -u 5 -a 1 -o 0.1 -p assembly_golden_path -i assembly_silver_path_all.fq -h 3 -j 14 -P 15 -d 5 -x10 -s 1011011110110111101101 -g 5e6 -b 10  -m 0 --verbose


...

/opt/conda/envs/goldrush_env/bin/share/ntlink-1.3.9-3/bin/ntlink_patch_gaps.py --path assembly_golden_path.goldpolish-polished.span2.dist500.tigmint.fa.k40.w250.z1000.ntLink.ntLink.ntLink.ntLink.gap_fill.fa.k40.w250.z1000.trimmed_scafs.path \
 --mappings assembly_golden_path.goldpolish-polished.span2.dist500.tigmint.fa.k40.w250.z1000.ntLink.ntLink.ntLink.ntLink.gap_fill.fa.k40.w250.z1000.verbose_mapping.tsv -s assembly_golden_path.goldpolish-polished.span2.dist500.tigmint.fa.k40.w250.z1000.ntLink.ntLink.ntLink.ntLink.gap_fill.fa --reads Epi300_RC.qc.fastq -o assembly_golden_path.goldpolish-polished.span2.dist500.tigmint.fa.k40.w250.z1000.ntLink.ntLink.ntLink.ntLink.gap_fill.fa.k40.w250.z1000.ntLink.scaffolds.gap_fill.fa --large_k 40 --min_gap 1 \
 --trims assembly_golden_path.goldpolish-polished.span2.dist500.tigmint.fa.k40.w250.z1000.ntLink.ntLink.ntLink.ntLink.gap_fill.fa.k40.w250.z1000.trimmed_scafs.tsv -k 20 -w 10 
Running ntLink gap-filling...

Parameters:
	--path assembly_golden_

[2024-09-27 17:04:22][WARNING] assembly_golden_path.goldpolish-polished.span2.dist500.tigmint.fa.k40.w250.z1000.ntLink.ntLink.ntLink.ntLink.gap_fill.fa.k40.w250.z1000.ntLink.scaffolds.gap_fill.fa.scaffolds.masked_temp.fa is empty.
[2024-09-27 17:04:22][WARNING] assembly_golden_path.goldpolish-polished.span2.dist500.tigmint.fa.k40.w250.z1000.ntLink.ntLink.ntLink.ntLink.gap_fill.fa.k40.w250.z1000.ntLink.scaffolds.gap_fill.fa.reads.masked_temp.fa is empty.


rm -f assembly_golden_path.goldpolish-polished.span2.dist500.tigmint.fa.k40.w250.z1000.ntLink.ntLink.ntLink.ntLink.gap_fill.fa.k40.w250.tsv
rm -f assembly_golden_path.goldpolish-polished.span2.dist500.tigmint.fa.k40.w250.z1000.ntLink.ntLink.ntLink.ntLink.gap_fill.fa.k40.w250.z1000.trimmed_scafs.fa
rm -f assembly_golden_path.goldpolish-polished.span2.dist500.tigmint.fa.k40.w250.z1000.ntLink.ntLink.ntLink.ntLink.gap_fill.fa.k40.w250.z1000.trimmed_scafs.tsv
rm -f assembly_golden_path.goldpolish-polished.span2.dist500.tigmint.fa.k40.w250.z1000.ntLink.ntLink.ntLink.ntLink.gap_fill.fa.k40.w250.z1000.stitch.path
rm -f assembly_golden_path.goldpolish-polished.span2.dist500.tigmint.fa.k40.w250.z1000.ntLink.ntLink.ntLink.ntLink.gap_fill.fa.k40.w250.z1000.stitch.abyss-scaffold.fa
rm -f assembly_golden_path.goldpolish-polished.span2.dist500.tigmint.fa.k40.w250.z1000.ntLink.ntLink.ntLink.ntLink.gap_fill.fa.k40.w250.z1000.trimmed_scafs.path
rm -f assembly_golden_path.goldpolish-polished.span2.dist50

INFO:    Cleanup error: while unmounting /home/tony/lib/miniforge3/envs/epi300asm/var/apptainer/mnt/session/underlay/usr directory: device or resource busy, while unmounting /home/tony/lib/miniforge3/envs/epi300asm/var/apptainer/mnt/session/underlay/opt directory: device or resource busy, while unmounting /home/tony/lib/miniforge3/envs/epi300asm/var/apptainer/mnt/session/final directory: device or resource busy


0

In [25]:
from Bio import SeqIO
ASM = ASSEMBLY_DIR/f"{epi300}.fna"

with open(ASM, "w") as f:
    seqs = []
    for e in SeqIO.parse(ASSEMBLY_DIR/f"{epi300}.goldrush.fna", "fasta"):
        header = f"length={len(e.seq)} goldrush={e.description}"
        seqs.append(e)
    seqs = sorted(seqs, key=lambda x: len(x.seq), reverse=True)
    for i, e in enumerate(seqs):
        header = f"C{i+1} length={len(e.seq)} goldrush={e.description}"
        f.write(f">{header}\n{e.seq}\n")

In [26]:
FASTQ_NO_QC = WORKSPACE_ROOT/"data/work/Epi300_RC.fastq"
if not FASTQ_NO_QC.exists():
    os.system(f"""\
        pigz -dc {WORKSPACE_ROOT}/data/Epi300_RC.fastq.gz > {FASTQ_NO_QC}
    """)

In [27]:
import stat

read_alignment = WORKSPACE_ROOT/"lib/containers/read_alignment.sif"
quast = WORKSPACE_ROOT/"lib/containers/quast.sif"
out = WORKSPACE_ROOT/"data/assembly_qc"
WS = WORKSPACE_ROOT/"data"
READ_FILES = [FASTQ_NO_QC.relative_to(WS)]
cov_tsv = WS/f"{epi300}.coverage.tsv"
stats_tsv = WS/f"{epi300}.stats.txt"
bounce = WS/"work/minimap_bounce.sh"

# https://lh3.github.io/minimap2/minimap2.html
# minimap2 options:
# -x sr                 makes alignment super strict
# --sr                  Enable short-read alignment heuristics, more sensitivity
# -2                    use two io threads, more peak memory
# -a                    SAM format
# --secondary=no        Whether to output secondary alignments [no]
# --sam-hit-only 	    In SAM, don’t output unmapped reads. !this results in report saying 100% reads mapped!
# --heap-sort=no|yes    Heap merge is faster for short reads, but slower for long reads. [no]
# --frag=yes

        # minimap2 -a --sr -2 -t {cpus} --secondary=no \
        #     --heap-sort=yes --frag=yes \
with open(bounce, "w") as f:
    f.write(f"""\
        cd /ws
        BAM=./temp.coverage.bam
        minimap2 -x map-hifi -a -2 -t {cpus} \
            {ASSEMBLY_DIR.name}/{ASM.name} {' '.join([str(f) for f in READ_FILES])} | samtools sort -o $BAM --write-index -
        
        bedtools genomecov -ibam $BAM -bg >/ws/{cov_tsv.name}
        samtools flagstat $BAM >/ws/{stats_tsv.name}
        rm ./temp*
    """)
os.chmod(bounce, stat.S_IRWXU|stat.S_IRWXG|stat.S_IRWXO)

os.system(f"""\
    cd {WS}

    date
    echo "coverage"
    singularity run -B ./:/ws {read_alignment} \
        /ws/work/{bounce.name}

    date
    echo "aggregate coverage"
""")


contig2length = {}
with open(ASM) as fa:
        current = None
        length = 0
        def _submit():
            contig2length[current] = length
        for l in fa:
            if l[0] == ">":
                if current is not None: _submit()
                current = l[1:-1].split(" ")[0]
                length = 0
            else:
                length += len(l)-1 # minus 1 for "\n"
        _submit()
agg_out = f"{epi300}.cov_per_contig.tsv"
with open(WS/cov_tsv) as f:
    with open(WS/agg_out, "w") as of:
        last_k = None
        entry = []
        def _submit():
            global entry
            total = contig2length[last_k]
            c = 0.0
            for span, val in entry:
                c += (span/total)*val
            # assume no overlap, so total == total span of contig
            of.write("\t".join(str(x) for x in [last_k, c, total])+"\n")
            entry = []

        for l in f:
            k, s, e, val = l[:-1].split("\t")
            s, e, val = [int(x) for x in [s, e, val]]
            if k != last_k:
                if last_k is not None: _submit()
                last_k = k
            entry.append((e-s, val))
        _submit()

os.system(f"""\
    cd {WS}
    mkdir -p {out}
    mv ./{agg_out} {out}/
    mv ./{cov_tsv.name} {out}/
    mv ./{stats_tsv.name} {out}/

    date
    echo "quast"
    singularity run -B ./:/ws {quast} \
    quast -t {cpus} \
        -o /ws/quast \
        /ws/{ASSEMBLY_DIR.name}/{ASM.name}
    cp ./quast/transposed_report.tsv {out}/{epi300}.quast.tsv
    tar -cf - ./quast | pigz -7 -p {cpus} >{out}/{epi300}.quast.tar.gz
    rm -r ./quast
""")

Fri Sep 27 19:47:46 PDT 2024
coverage


[M::mm_idx_gen::0.091*1.03] collected minimizers
[M::mm_idx_gen::0.114*1.43] sorted minimizers
[M::main::0.114*1.43] loaded/built the index for 42 target sequence(s)
[M::mm_mapopt_update::0.121*1.40] mid_occ = 50
[M::mm_idx_stat] kmer size: 19; skip: 19; is_hpc: 0; #seq: 42
[M::mm_idx_stat::0.127*1.39] distinct minimizers: 446598 (94.26% are singletons); average occurrences: 1.083; average spacing: 10.002; total length: 4837344
[M::worker_pipeline::33.084*12.76] mapped 36909 sequences
[M::worker_pipeline::60.849*13.34] mapped 38424 sequences
[M::worker_pipeline::82.838*13.46] mapped 42881 sequences
[M::worker_pipeline::103.701*13.63] mapped 35903 sequences
[M::worker_pipeline::126.349*13.74] mapped 36900 sequences
[M::worker_pipeline::146.296*13.79] mapped 38519 sequences
[M::worker_pipeline::155.684*13.34] mapped 35754 sequences
[M::main] Version: 2.26-r1175
[M::main] CMD: minimap2 -x map-hifi -a -2 -t 14 assembly/epi300.fna work/Epi300_RC.fastq
[M::main] Real time: 155.698 sec; CPU: 

Fri Sep 27 19:52:16 PDT 2024
aggregate coverage
Fri Sep 27 19:52:18 PDT 2024
quast
/opt/conda/bin/quast -t 14 -o /ws/quast /ws/assembly/epi300.fna

Version: 5.0.2

System information:
  OS: Linux-5.15.153.1-microsoft-standard-WSL2-x86_64-with-debian-10.6 (linux_64)
  Python version: 3.7.10
  CPUs number: 16

Started: 2024-09-27 19:52:19

Logging to /ws/quast/quast.log

CWD: /home/tony/workspace/projects/epi300_assembly/data
Main parameters: 
  MODE: default, threads: 14, minimum contig length: 500, minimum alignment length: 65, \
  ambiguity: one, threshold for extensive misassembly size: 1000

Contigs:
  Pre-processing...
  /ws/assembly/epi300.fna ==> epi300

2024-09-27 19:52:20
Running Basic statistics processor...
  Contig files: 
    epi300
  Calculating N50 and L50...
    epi300, N50 = 4553367, L50 = 1, Total length = 4836284, GC % = 50.77, # N's per 100 kbp =  0.12
  Drawing Nx plot...
    saved to /ws/quast/basic_stats/Nx_plot.pdf
  Drawing cumulative plot...
    saved to /ws/qu

0

In [28]:
os.system(f"""\
    cd {WS}
    mkdir -p {out}
    mv ./{agg_out} {out}/
    mv ./{cov_tsv.name} {out}/
    mv ./{stats_tsv.name} {out}/

    date
    echo "quast"
    singularity run -B ./:/ws {quast} \
    quast -t {cpus} \
        -o /ws/quast \
        /ws/{ASSEMBLY_DIR.name}/{ASM.name}
    cp ./quast/transposed_report.tsv {out}/{epi300}.quast.tsv
    tar -cf - ./quast | pigz -7 -p {cpus} >{out}/{epi300}.quast.tar.gz
    rm -r ./quast
""")

mv: cannot stat './epi300.cov_per_contig.tsv': No such file or directory
mv: cannot stat './epi300.coverage.tsv': No such file or directory
mv: cannot stat './epi300.stats.txt': No such file or directory


Fri Sep 27 19:52:22 PDT 2024
quast
/opt/conda/bin/quast -t 14 -o /ws/quast /ws/assembly/epi300.fna

Version: 5.0.2

System information:
  OS: Linux-5.15.153.1-microsoft-standard-WSL2-x86_64-with-debian-10.6 (linux_64)
  Python version: 3.7.10
  CPUs number: 16

Started: 2024-09-27 19:52:22

Logging to /ws/quast/quast.log

CWD: /home/tony/workspace/projects/epi300_assembly/data
Main parameters: 
  MODE: default, threads: 14, minimum contig length: 500, minimum alignment length: 65, \
  ambiguity: one, threshold for extensive misassembly size: 1000

Contigs:
  Pre-processing...
  /ws/assembly/epi300.fna ==> epi300

2024-09-27 19:52:23
Running Basic statistics processor...
  Contig files: 
    epi300
  Calculating N50 and L50...
    epi300, N50 = 4553367, L50 = 1, Total length = 4836284, GC % = 50.77, # N's per 100 kbp =  0.12
  Drawing Nx plot...
    saved to /ws/quast/basic_stats/Nx_plot.pdf
  Drawing cumulative plot...
    saved to /ws/quast/basic_stats/cumulative_plot.pdf
  Drawing GC

0